# Training and deploying a tabular model using Vertex custom training job

![Training pipeline](../images/custom-tabular.png)

## Import the required packages

In [1]:
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions


from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io import bigquery as tfio_bq


## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites and configure GCP authentication.*

In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-vertex-workshop-bucket'
VERTEX_SA = 'vertex-sa@jk-mlops-dev.iam.gserviceaccount.com'

## Initialize Vertex AI SDK

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

## Create or set Tensorboard

In [ ]:
tb_client = api_client = vertex_ai.initializer.global_config.create_client(
        client_class=vertex_ai.utils.TensorboardClientWithOverride, location_override=REGION
)
parent = f'projects/{PROJECT}/locations/{REGION}'

tensorboard_display_name = 'Workshop Tensorboard'
tensorboard_ref = None

for tensorboard in tb_client.list_tensorboards(parent=parent):
    if tensorboard.display_name == tensorboard_display_name:
        tensorboard_ref = tensorboard
        
if not tensorboard_ref:
    print('Creating new Tensorboard')
    tb_specs = types.Tensorboard(
        display_name=tensorboard_display_name,
        description=tensorboard_display_name
    )
    operation = tb_client.create_tensorboard(parent=parent, tensorboard=tb_specs)
    tensorboard_ref = operation.result()
else:
    print('Using existing Tensorboard:', tensorboard_ref.name)

## Prepare training data in BigQuery

### Explore Chicago Taxi dataset

In [ ]:
%%bigquery 

SELECT 
    *
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
LIMIT 5

In [ ]:
%%bigquery data

SELECT 
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek, 
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2015 
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek

In [ ]:
data

In [ ]:
data.plot(kind='bar', x='trip_dayname', y='trip_count')

### Creating training and validation splits

In [4]:
BQ_DATASET_NAME = 'training_dataset' # Change to your BQ datasent name.
BQ_TABLE_NAME = 'training_table'
BQ_LOCATION = 'US'

#### Create a training dataset

In [5]:
client = bigquery.Client()

dataset_id = f'{PROJECT}.{BQ_DATASET_NAME}'
dataset = bigquery.Dataset(dataset_id)
dataset.location = BQ_LOCATION

try:
    dataset = client.create_dataset(dataset, timeout=30)
    print('Created dataset: ', dataset_id)
except exceptions.Conflict:
    print('Dataset {} already exists'.format(dataset_id))

Dataset jk-mlops-dev.training_dataset already exists


#### Create a training table

In [ ]:
sample_size = 1000000
year = 2020

sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        FORMAT_DATETIME('%Y-%d-%m', trip_start_timestamp) AS date,
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      CASE (ABS(MOD(FARM_FINGERPRINT(date),10))) 
          WHEN 9 THEN 'testing'
          WHEN 8 THEN 'validation'
          ELSE 'training' END AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

sql_script = sql_script.replace(
    '@PROJECT', PROJECT).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TABLE', BQ_TABLE_NAME).replace(
    '@YEAR', str(year)).replace(
    '@LIMIT', str(sample_size))


In [ ]:
job = client.query(sql_script)
job.result()

In [ ]:
sql_script = f'''
SELECT * 
FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME} 
WHERE data_split='training'
'''
training_data = client.query(sql_script).result().to_dataframe()

In [ ]:
training_data.head(3)

In [ ]:
sql_script = f'''
SELECT ARRAY(SELECT DISTINCT payment_type FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}) as payment_type_vocab,
       ARRAY(SELECT DISTINCT pickup_grid FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}) as pickup_grid_vocab,
       ARRAY(SELECT DISTINCT dropoff_grid FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}) as dropoff_grid_vocab,
       (SELECT AS STRUCT AVG(trip_seconds), VARIANCE(trip_seconds) FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}) as trip_seconds_stats,
'''

training_data_stats = client.query(sql_script).result().to_dataframe()
training_data_stats.T

In [ ]:
sql_script = f'''
SELECT * EXCEPT (trip_start_timestamp, data_split)
FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME} 
'''
sample_data = client.query(sql_script).result().to_dataframe()

In [ ]:
sample_data.head().T

In [ ]:
sample_data.tip_bin.value_counts()

In [ ]:
sample_data.euclidean.hist()

In [ ]:
sql_script = f'''
SELECT * 
FROM {PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME} 
WHERE data_split='training'
'''
training_data = client.query(sql_script).result().to_dataframe()

In [ ]:
training_data.data_split.value_counts()

In [ ]:
training_data.nunique()

## Prepare the model

In [10]:
tfio_bq_client = tfio_bq.BigQueryClient()

In [41]:
TARGET_FEATURE_NAME = "tip_bin"

TARGET_LABELS = ["tip<20%", "tip>=20%"]


FEATURES = {
    "tip_bin": ("categorical", dtypes.int32),
    "trip_month": ("categorical", dtypes.int32),
    "trip_day": ("categorical", dtypes.int32),
    "trip_day_of_week": ("categorical", dtypes.int32),
    "trip_hour": ("categorical", dtypes.int32),
    "payment_type": ("categorical", dtypes.string),
    "pickup_grid": ("categorical", dtypes.string),
    "dropoff_grid": ("categorical", dtypes.string),
    "euclidean": ("numeric", dtypes.double),
    "trip_seconds": ("numeric", dtypes.double),
    "trip_miles": ("numeric", dtypes.double),
}

FEATURES = {
    "tip_bin": ("categorical", dtypes.int64),
    "trip_month": ("categorical", dtypes.int64),
    "trip_day": ("categorical", dtypes.int64),
    "trip_day_of_week": ("categorical", dtypes.int64),
    "trip_hour": ("categorical", dtypes.int64),
    "payment_type": ("categorical", dtypes.string),
    "pickup_grid": ("categorical", dtypes.string),
    "dropoff_grid": ("categorical", dtypes.string),
    "euclidean": ("numeric", dtypes.double),
    "trip_seconds": ("numeric", dtypes.int64),
    "trip_miles": ("numeric", dtypes.double),
}


In [42]:
selected_fields, output_types = zip(*[(key, value[1]) for key, value in FEATURES.items()])

print(selected_fields)
print(output_types)

('tip_bin', 'trip_month', 'trip_day', 'trip_day_of_week', 'trip_hour', 'payment_type', 'pickup_grid', 'dropoff_grid', 'euclidean', 'trip_seconds', 'trip_miles')
(tf.int64, tf.int64, tf.int64, tf.int64, tf.int64, tf.string, tf.string, tf.string, tf.float64, tf.int64, tf.float64)


In [43]:
selected_fields = {key: {'output_type': value[1]} for key, value in FEATURES.items()}
selected_fields

{'tip_bin': {'output_type': tf.int64},
 'trip_month': {'output_type': tf.int64},
 'trip_day': {'output_type': tf.int64},
 'trip_day_of_week': {'output_type': tf.int64},
 'trip_hour': {'output_type': tf.int64},
 'payment_type': {'output_type': tf.string},
 'pickup_grid': {'output_type': tf.string},
 'dropoff_grid': {'output_type': tf.string},
 'euclidean': {'output_type': tf.float64},
 'trip_seconds': {'output_type': tf.int64},
 'trip_miles': {'output_type': tf.float64}}

In [44]:
parent = f'projects/{PROJECT}'
#selected_fields = ['trip_seconds', 'trip_miles']
output_types = [dtypes.int64, dtypes.double]


read_session = tfio_bq_client.read_session(
    parent=parent,
    project_id=PROJECT,
    table_id=BQ_TABLE_NAME,
    dataset_id=BQ_DATASET_NAME,
    selected_fields=selected_fields,
    #output_types=output_types
)

dataset = read_session.parallel_read_rows()

In [45]:
iterator = iter(dataset)

In [46]:
next(iterator)

OrderedDict([('dropoff_grid',
              <tf.Tensor: shape=(), dtype=string, numpy=b'POINT(-87.6 41.8)'>),
             ('euclidean',
              <tf.Tensor: shape=(), dtype=float64, numpy=10666.168994769989>),
             ('payment_type',
              <tf.Tensor: shape=(), dtype=string, numpy=b'Unknown'>),
             ('pickup_grid',
              <tf.Tensor: shape=(), dtype=string, numpy=b'POINT(-87.7 41.9)'>),
             ('tip_bin', <tf.Tensor: shape=(), dtype=int64, numpy=0>),
             ('trip_day', <tf.Tensor: shape=(), dtype=int64, numpy=19>),
             ('trip_day_of_week', <tf.Tensor: shape=(), dtype=int64, numpy=6>),
             ('trip_hour', <tf.Tensor: shape=(), dtype=int64, numpy=0>),
             ('trip_miles', <tf.Tensor: shape=(), dtype=float64, numpy=0.4>),
             ('trip_month', <tf.Tensor: shape=(), dtype=int64, numpy=6>),
             ('trip_seconds', <tf.Tensor: shape=(), dtype=int64, numpy=900>)])

In [ ]:
for record in dataset.take(5):
    print(record)

## Create a Vertex managed tabular dataset

### List existing tabular datasets

In [ ]:
display_name = 'Chicago Taxi Tips'
filter = 'display_name="{}"'.format(display_name)

for dataset in vertex_ai.TabularDataset.list(filter=filter):
    print(dataset.display_name, ' : ', dataset.resource_name)

### Create a new tabular dataset based on the BigQuery table

In [ ]:
bq_source = f'bq://{PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}'


dataset = vertex_ai.TabularDataset.create(
    display_name=display_name, bq_source=bq_source
)

## Prepare a training script

In [ ]:
folder = 'trainer'
if tf.io.gfile.exists(folder):
    tf.io.gfile.rmtree(folder)
tf.io.gfile.mkdir(folder)
file_path = os.path.join(folder, 'train.py')

In [ ]:
%%writefile {file_path}


# Copyright 2021 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import tensorflow as tf
from tensorflow import keras

from absl import app
from absl import flags
from absl import logging

FEATURE_NAMES = [
    "trip_month",
    "trip_day",
    "trip_day_of_week",
    "trip_hour",
    "trip_seconds",
    "trip_miles",
    "payment_type",
    "pickup_grid",
    "dropoff_grid",
    "euclidean",
    "loc_cross",
]

TARGET_FEATURE_NAME = "tip_bin"

TARGET_LABELS = ["tip<20%", "tip>=20%"]

NUMERICAL_FEATURE_NAMES = [
    "trip_seconds",
    "trip_miles",
    "euclidean",
]

EMBEDDING_CATEGORICAL_FEATURES = {
    "trip_month": 2,
    "trip_day": 4,
    "trip_hour": 3,
    "pickup_grid": 3,
    "dropoff_grid": 3,
    "loc_cross": 10,
}

ONEHOT_CATEGORICAL_FEATURE_NAMES = ["payment_type", "trip_day_of_week"]

def create_model_inputs():
    inputs = {}
    for name in NUMERICAL_FEATURE_NAMES:
        inputs[name] = keras.layers.Input(name=name, shape=[], dtype=tf.float32)
    for name in list(EMBEDDING_CATEGORICAL_FEATURES.keys()) + ONEHOT_CATEGORICAL_FEATURE_NAMES:
        inputs[name] = keras.layers.Input(name=name, shape=[], dtype=tf.int64)
    return inputs


def create_binary_classifier(feature_vocab_sizes, hyperparams):
    input_layers = create_model_inputs()

    layers = []
    for feature_name in input_layers.keys:
        if feature_name in EMBEDDING_CATEGORICAL_FEATURES:
            vocab_size = feature_vocab_sizes[feature_name]
            embedding_size = EMBEDDING_CATEGORICAL_FEATURES[feature_name]
            embedding_output = keras.layers.Embedding(
                input_dim=vocab_size + 1,
                output_dim=embedding_size,
                name=f"{feature_name}_embedding",
            )(input_layers[feature_name])
            layers.append(embedding_output)
        elif feature_name in ONEHOT_CATEGORICAL_FEATURE_NAMES:
            vocab_size = feature_vocab_sizes[feature_name]
            onehot_layer = keras.layers.experimental.preprocessing.CategoryEncoding(
                max_tokens=vocab_size,
                output_mode="binary",
                name=f"{feature_name}_onehot",
            )(input_layers[feature_name])
            layers.append(onehot_layer)
        elif feature_name in NUMERICAL_FEATURE_NAMES:
            numeric_layer = tf.expand_dims(input_layers[feature_name], -1)
            layers.append(numeric_layer)
        else:
            pass

    joined = keras.layers.Concatenate(name="combines_inputs")(layers)
    feedforward_output = keras.Sequential(
        [
            keras.layers.Dense(units, activation="relu")
            for units in hyperparams["hidden_units"]
        ],
        name="feedforward_network",
    )(joined)
    logits = keras.layers.Dense(units=1, name="logits")(feedforward_output)

    model = keras.Model(inputs=input_layers, outputs=[logits])
    return model


def main(argv):
    del argv
    
    inputs = create_model_inputs()
    print(inputs)

if __name__ == '__main__':
    logging.set_verbosity(logging.INFO)
    app.run(main)